In [ ]:
import os
import pandas as pd
import numpy as np
from neo4j import Query, GraphDatabase, RoutingControl, Result
from graphdatascience import GraphDataScience 

In [ ]:
csv = pd.read_csv('https://raw.githubusercontent.com/deshanadesai/Symptom-X-/master/dataset_clean1.csv')
csv.dropna().shape

## Database connection

In [6]:
DB_ULR = "neo4j+s://2bdf62cf.databases.neo4j.io"
DB_USER = "neo4j"
DB_PASS = "qK2qvX0bVb_l0a5W9bsRmL57NVYuLGZ5Gbm78eN3cAw"
DB_NAME = "neo4j" # Have to be neo4j for neo4j aura (but keep it, good for testing on local dev env)

In [12]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))
driver.verify_connectivity()

In [18]:
auth = ("neo4j", "qK2qvX0bVb_l0a5W9bsRmL57NVYuLGZ5Gbm78eN3cAw")
url = "neo4j+s://2bdf62cf.databases.neo4j.io"
gds = GraphDataScience(url, auth=auth, aura_ds=False)
gds.set_database(database=DB_NAME)

UnableToConnectError: {code: Neo.ClientError.Statement.SyntaxError} {message: Unknown function 'gds.version' (line 1, column 8 (offset: 7))
"RETURN gds.version()"
        ^}

In [ ]:
data=driver.execute_query(
    '''
        match (c:Customer)
        return c.wcc as currentCommunity, count(*) as size
    ''',
    database_=DB_NAME,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)
data.hist("score",bins=10)

## Utility functions

In [ ]:
## Utility
def split_dataframe(df, chunk_size = 5000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

## Define indexes and constraints

In [ ]:
schema_statements = [
    'create constraint if not exists for (n:Disease) require (n.name) is node key',
    'create constraint if not exists for (n:Disease) require (n.name) is ::String',
    'create constraint if not exists for (n:Symptom) require (n.name) is node key',
    'create constraint if not exists for (n:Symptom) require (n.name) is ::String'
]
for statement in schema_statements:
    driver.execute_query(
        statement,
        database_=DB_NAME,
        routing_=RoutingControl.WRITE
    )

# Fetch all constraints
schema_result_df  = driver.execute_query(
    'show constraints',
    database_=DB_NAME,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)
schema_result_df.head(100)

In [ ]:
driver.execute_query(
        '''
        unwind $data as row
        with row where row['Source'] is not null
        merge (d:Disease{name:row['Source']})
        merge (s:Symptom{name:row['Target']})
        merge (s)-[:SYMPTOM_OF]->(d)
        ''',
        database_=DB_NAME,
        data = csv.dropna().to_dict('records'),
        routing_=RoutingControl.WRITE
    )

In [ ]:
p1,_ = gds.graph.project(
    'p1',
    ['Symptom', 'Disease'],
    ['SYMPTOM_OF']
)

In [ ]:
p1

In [ ]:
gds.nodeSimilarity.stats(p1)

In [ ]:
gds.nodeSimilarity.write(p1, similarityMetric='OVERLAP', similarityCutoff=0.7, 
                         writeRelationshipType='NARROWER_THAN', writeProperty='score')